In [1]:
import os
os.chdir("C:/Users/zetru/OneDrive/Mémoire python/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['axes.grid'] = True
import seaborn as sns
sns.set_theme(style="whitegrid")

In [2]:
df=pd.read_hdf('df_v5.h5','df_v5')
df

,cp,id,nom_carburant,id_carburant,maj,valeur,litre_brut,type,latitude,longitude,ville,stations_rayon_5km,competition_geo
0,01,1000001,Gazole,1,2022-01-03,0.702500,0.435626,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
1,01,1000001,Gazole,1,2022-01-05,0.692500,0.447949,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
2,01,1000001,Gazole,1,2022-01-10,0.728333,0.452874,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
3,01,1000001,Gazole,1,2022-01-17,0.739167,0.484043,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
4,01,1000001,Gazole,1,2022-01-21,0.769167,0.497616,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4537444,95,95870010,SP98,6,2022-12-23,0.856233,0.488413,R,48.936,2.206,Bezons,15,4
4537445,95,95870010,SP98,6,2022-12-27,0.874567,0.487324,R,48.936,2.206,Bezons,15,4
4537446,95,95870010,SP98,6,2022-12-28,0.874567,0.484175,R,48.936,2.206,Bezons,15,4
4537447,95,95870010,SP98,6,2022-12-29,0.879567,0.477600,R,48.936,2.206,Bezons,15,4


In [6]:
df.columns

Index(['cp', 'id', 'nom_carburant', 'id_carburant', 'maj', 'valeur',
       'litre_brut', 'type', 'latitude', 'longitude', 'ville',
       'stations_rayon_5km', 'competition_geo'],
      dtype='object')

In [20]:
prix_quotidien_moyen = df.groupby(['maj', 'nom_carburant', 'type'])['valeur'].mean().reset_index()

prix_quotidien_moyen['maj'] = pd.to_datetime(prix_quotidien_moyen['maj'])
prix_quotidien_moyen['année'] = prix_quotidien_moyen['maj'].dt.year

prix_annuel_moyen = prix_quotidien_moyen.groupby(['année', 'type', 'nom_carburant'])['valeur'].mean().reset_index()
prix_brut_quotidien = df.groupby('maj')['litre_brut'].mean().reset_index()
prix_brut_quotidien['maj'] = pd.to_datetime(prix_brut_quotidien['maj'])
prix_brut_quotidien['année'] = prix_brut_quotidien['maj'].dt.year
prix_brut_annuel_moyen = prix_brut_quotidien.groupby('année')['litre_brut'].mean().reset_index()
prix_annuel_moyen = prix_annuel_moyen.merge(prix_brut_annuel_moyen, on='année', how='left')
prix_annuel_moyen['taux_de_marge'] = 1 + (prix_annuel_moyen['valeur'] - prix_annuel_moyen['litre_brut']) / prix_annuel_moyen['litre_brut']

resultat_final = prix_annuel_moyen.pivot_table(index=['nom_carburant'], columns='type', values='taux_de_marge')
resultat_final = resultat_final[['R', 'A']]
resultat_final.columns = ['Route', 'Autoroute']
resultat_final

,Route,Autoroute
nom_carburant,,
E85,1.125631,1.187540
GPLc,1.017049,1.165851
Gazole,1.761338,1.917962
SP95,1.580707,1.604844
SP95-E10,1.535205,1.710471
SP98,1.642812,1.823711


In [21]:
resultat_final.to_latex('tex/tx_de_marge.tex')